# Conversational Agent (LLM + Code)

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain.tools import tool

In [3]:
import requests
from pydantic import BaseModel, Field
from datetime import datetime

# Input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(description="Latitude of the location to fetch weather data")
    longitude: float = Field(description="Longitude of the location to fetch weather data")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.now()
    time_list = [datetime.fromisoformat(time_str.replace("Z", "+00:00")) for time_str in results["hourly"]["time"]]
    temperature_list = results["hourly"]["temperature_2m"]
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f"{current_temperature}°C"

In [4]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia Search and get page summaries"""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[:3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary.strip()}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DIsambiguationError,
        ):
            pass
    
    if not summaries:
        return "No good Wikipedia Search Result was found"

    return "\n\n".join(summaries)

In [5]:
tools = [get_current_temperature, search_wikipedia]

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [ ]:
from langchain.prompts import MessagesPlaceholder

functions = [convert_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"), # used to pass the history of tools that are called
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()
result = chain.invoke({
    "input": "What is the weather in Douala?",
    "agent_scratchpad": [], # no action taken for the moment
})
type(result), result.tool, result.tool_input

(langchain_core.agents.AgentActionMessageLog,
 'get_current_temperature',
 {'latitude': 4.0511, 'longitude': 9.7679})

In [8]:
from langchain.agents.format_scratchpad import format_to_openai_functions

observation = get_current_temperature.invoke(result.tool_input)
result = chain.invoke({
    "input": "What is the weather in Douala?", 
    "agent_scratchpad": format_to_openai_functions([(result, observation)]) # append function params and result
})
type(result), result.return_values["output"]

(langchain_core.agents.AgentFinish,
 'The current temperature in Douala is 25.1°C.')

## Encapsulate everything in a function

In [9]:
from langchain.schema.runnable import RunnablePassthrough # its assign method creates a new key in the dictionary that is passed

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [10]:
from langchain.schema.agent import AgentFinish

def run_agent(user_input):
    intermediate_steps = [] # agent actions and observations
    
    while True:
        result = agent_chain.invoke({
            "input": user_input,
            # "agent_scratchpad": format_to_openai_functions(intermediate_steps) # before
            "intermediate_steps": intermediate_steps # the agent_scratchpad is now created by the RunnablePassThrough
        })
        
        if isinstance(result, AgentFinish):
            return result
        
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        
        observation = tool.invoke(result.tool_input)        
        intermediate_steps.append((result, observation))

In [11]:
run_agent("What's the weather in Douala?")

AgentFinish(return_values={'output': 'The current temperature in Douala is 25.1°C.'}, log='The current temperature in Douala is 25.1°C.')

In [12]:
run_agent("What's World War II?")

AgentFinish(return_values={'output': 'World War II, also known as the Second World War, was a global conflict that took place from September 1, 1939, to September 2, 1945. It involved two coalitions: the Allies and the Axis powers. The war resulted in significant devastation and loss of life, with an estimated 70 to 85 million deaths, including civilians and military personnel.\n\nKey events leading up to World War II included the rise of fascism in Europe, militarism in Japan, and unresolved tensions from the aftermath of World War I. The war began when Nazi Germany invaded Poland in 1939, leading to declarations of war by the United Kingdom and France against Germany.\n\nThe war involved major battles and campaigns in Europe, Asia, and the Pacific. The Allies, including countries like the United States, the Soviet Union, and the United Kingdom, eventually emerged victorious over the Axis powers, which included Germany, Japan, and Italy.\n\nWorld War II had a profound impact on the po

In [13]:
run_agent("Hi")

AgentFinish(return_values={'output': 'Hello! How can I assist you today?'}, log='Hello! How can I assist you today?')

In [14]:
from langchain.agents import AgentExecutor # adds better logging, error handling, etc.

agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [15]:
agent_executor.invoke({"input": "What's the weather in Douala?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 4.0511, 'longitude': 9.7679}`


25.1°CThe current temperature in Douala is 25.1°C.

> Finished chain.


{'input': "What's the weather in Douala?",
 'output': 'The current temperature in Douala is 25.1°C.'}

In [16]:
# agent_executor.invoke({"input": "What's WW2?"})

## Add chat history

In [17]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: convert_to_openai_function(x["intermediate_steps"]) if len(x["intermediate_steps"]) > 0 else []
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
from langchain.memory import ChatMessageHistory # keep a list of messages in the memory
from langchain_core.runnables.history import RunnableWithMessageHistory

session_id = "test"
memory = ChatMessageHistory(session_id=session_id)

agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [19]:
agent_with_chat_history.invoke(
    {"input": "My name is Ngannou"},
    {"configurable": {"session_id": session_id}},
)



> Entering new AgentExecutor chain...


Hello Ngannou! How can I assist you today?

> Finished chain.


{'input': 'My name is Ngannou',
 'chat_history': [],
 'output': 'Hello Ngannou! How can I assist you today?'}

In [ ]:
agent_with_chat_history.invoke(
    {"input": "What's my name"},
    {"configurable": {"session_id": session_id}},
)



> Entering new AgentExecutor chain...
Your name is Ngannou. How can I assist you today, Ngannou?

> Finished chain.


{'input': "What's my name",
 'chat_history': [HumanMessage(content='My name is Ngannou', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ngannou! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Ngannou. How can I assist you today, Ngannou?'}